# Part 0: Dataloader and Visualizations

In [1]:
import numpy as np
from PIL import Image
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import wandb

from voc_dataset import VOCDataset
from utils import *

USE_WANDB = False

/home/ec2-user/anaconda3/envs/vlr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q0.1: Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself and we have completed this part for you in the starter code. 

Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf). Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals for you (you downloaded this in the data preparation step).

Your task is to change the dataloader to obtain the proposed bounding boxes for each image. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
dataset = VOCDataset('trainval', top_n=10)

**Q0.1**: Load the image corresponding to index 2020 and print the GT labels associated with it.

**Hint**: items at a particular index can be accesed by usual indexing notation (dataset[idx])

In [3]:
# TODO: get the image information from index 2020
idx = 2020
result =  dataset[idx]
print("result is", result['rois']['top_boxes'])
print(dataset.get_class_name(result['gt_classes'][0]))

result is [[0.374, 0.002680965147453083, 0.958, 0.5898123324396782], [0.374, 0.5120643431635389, 0.762, 0.7855227882037533], [0.172, 0.002680965147453083, 0.962, 0.579088471849866], [0.372, 0.5120643431635389, 0.762, 0.6863270777479893], [0.514, 0.002680965147453083, 0.958, 0.514745308310992], [0.372, 0.4745308310991957, 0.726, 0.8900804289544236], [0.254, 0.3109919571045576, 0.726, 0.5817694369973191], [0.378, 0.48525469168900803, 0.8, 0.8605898123324397], [0.862, 0.002680965147453083, 1.0, 0.46380697050938335], [0.286, 0.49865951742627346, 0.764, 0.8284182305630027]]
train


## Q0.2 and Q0.3: Wandb Logging
First, let's initialize a Weights and Biases project. 

In [19]:
USE_WANDB = True
if USE_WANDB:
    wandb.init(project="vlr-hw1", reinit=True)

**Q0.2**: Complete this block for overlaying the ground truth box on an image.

**Hint**: convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions). You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [20]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))

# TODO: load the GT information corresponding to index 2020.
original_image = result['image']
gt_labels = result['gt_classes']
gt_boxes = result['gt_boxes']

img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

# TODO: log the GT bounding box
wandb.log({"bounding box": img})


SyntaxError: invalid syntax (2477532189.py, line 4)

**Q0.3**: Visualize the top 10 bounding proposals corresponding to index 2020.

**Hint**: Check the `get_box_data` function in `utils.py` and understand how it is being used. 

In [ ]:
# TODO: plot top ten proposals (of bounding boxes)
roi_boxes = result['rois']['top_boxes']
roi_labels = [0] * len(roi_boxes)

img_roi = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(roi_labels, roi_boxes)
    },
})
wandb.log({"top n roi boxes": img_roi})
